# Denoising Diffusion Probabilistic Models

[Denoising Diffusion Probabilistic Models](https://arxiv.org/pdf/2006.11239)

[Tutorial on Diffusion Models for Imaging and Vision](https://arxiv.org/pdf/2403.18103)

[Bayes theorem, the geometry of changing beliefs](https://youtu.be/HZGCoVF3YvM?si=wPw_XLl6pZFQmIws)

[Probabilities of probabilities](https://youtube.com/playlist?list=PLZHQObOWTQDOjmo3Y6ADm0ScWAlEXf-fp&si=2GUpUMdHkjuCqSPK)

[What are Diffusion Models?](https://youtu.be/fbLgFrlTnGU?si=_d1SbwC5wNQ6eVym)

[Diffusion Models | Paper Explanation | Math Explained](https://youtu.be/HoKDTa5jHvg?si=Df3zTMRsPOgloGvI)

[The Annotated Diffusion Model](https://huggingface.co/blog/annotated-diffusion)

[Generative Modeling by Estimating Gradients of the Data Distribution](https://yang-song.net/blog/2021/score/)

[What are Diffusion Models? Author: Lilian Weng](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/)

[How diffusion models work: the math from scratch](https://theaisummer.com/diffusion-models/)

[Step by Step visual introduction to Diffusion Models](https://erdem.pl/2023/11/step-by-step-visual-introduction-to-diffusion-models)


## Main components of DDPM

#### Forward Diffusion Process

The forward diffusion process is about adding noise gradually to the data so that it eventually becomes Gaussian noise. This can be expressed as a series of transitions, where at each step, a small amount of Gaussian noise is added.

#### Reverse Process (Denoising)

The reverse process is more complex and involves learning how to reverse the noise addition in order to recover the original data. Here, a neural network is typically trained to predict the noise added at each step.

#### Connection to Variational Inference (KL Divergence, ELBO, etc.)

This part of the model helps explain why and how we can learn the reverse process. We want the reverse process to approximate the true posterior distribution of the data (i.e., the reverse distribution). This can be done by minimizing the KL divergence between the true posterior and the learned reverse process.

Let’s start by walking through the **forward diffusion process**, where we gradually add noise to the data.

#### Forward Process (Diffusion of Data)

In the forward process, we transform data $\mathbf{x}_0$ (e.g., an image) into a sequence of noisy versions $\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_T$ through a gradual addition of Gaussian noise. By the end of the process (at time step $T$), the data should resemble pure noise, typically modeled as a Gaussian distribution $\mathcal{N}(0, I)$.

The forward process is defined as a Markov chain, where each state $\mathbf{x}_t$ depends only on the previous state $\mathbf{x}_{t-1}$. Specifically, we can express this step-wise transition as:

$$
q(\mathbf{x}_t | \mathbf{x}_{t-1}) = \mathcal{N}(\mathbf{x}_t; \sqrt{1-\beta_t} \mathbf{x}_{t-1}, \beta_t \mathbf{I})
$$

Here:
- $\mathbf{x}_t$ is the noisy data at time step $t$.
- $\beta_t$ is a variance schedule controlling the amount of noise added at each step.
- $\mathcal{N}(\mu, \Sigma)$ denotes a Gaussian distribution with mean $\mu$ and covariance $\Sigma$.

#### What's happening in each step?

At each step $t$, a small amount of Gaussian noise is added to the previous state $\mathbf{x}_{t-1}$. This noise is parameterized by $\beta_t$, which controls how much noise is injected at each step. The forward process moves the data from its original clean state $\mathbf{x}_0$ towards a fully noisy state $\mathbf{x}_T$.

- The term $\sqrt{1 - \beta_t} \mathbf{x}_{t-1}$ scales down the contribution of the clean data.
- The term $\beta_t \mathbf{I}$ introduces Gaussian noise.

By the time you reach $\mathbf{x}_T$, the data has been degraded into noise.

#### How do we accumulate noise?

We want to describe the relationship between $\mathbf{x}_0$ (the original data) and $\mathbf{x}_t$ at an arbitrary time step $t$. Since each step involves adding noise, we can compute the cumulative noise added over all the steps up to $t$.

By recursively applying the noise-adding process from $t = 1$ to $t = T$, we get:

$$
q(\mathbf{x}_t | \mathbf{x}_0) = \mathcal{N}(\mathbf{x}_t; \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t) \mathbf{I})
$$

Where:
- $\alpha_t = 1 - \beta_t$
- $\bar{\alpha}_t = \prod_{s=1}^{t} \alpha_s$

This equation tells us that $\mathbf{x}_t$ is a noisy version of the original data $\mathbf{x}_0$, where $\sqrt{\bar{\alpha}_t}$ is a decaying factor applied to the original data, and $1 - \bar{\alpha}_t$ governs the amount of noise accumulated over time.

#### Why is this form useful?

This equation is crucial because it allows us to directly sample $\mathbf{x}_t$ at any time step $t$ from the original data $\mathbf{x}_0$. The cumulative noise $\mathbf{x}_t$ is modeled as a Gaussian distribution where:
- The mean is $\sqrt{\bar{\alpha}_t} \mathbf{x}_0$, a scaled version of the original data.
- The variance is $(1 - \bar{\alpha}_t) \mathbf{I}$, which grows over time, introducing more noise.

#### What is $\bar{\alpha}_t$?

$\bar{\alpha}_t$ is the cumulative product of $\alpha_t$ values up to time step $t$. Recall that:

$$
\alpha_t = 1 - \beta_t
$$

where $\beta_t$ is the variance at each time step, which controls how much noise is added at step $t$.

Then, $\bar{\alpha}_t$ is defined as:

$$
\bar{\alpha}_t = \prod_{s=1}^{t} \alpha_s = \prod_{s=1}^{t} (1 - \beta_s)
$$

This represents the product of all the $\alpha$-values up to time step $t$.

#### Why does $\bar{\alpha}_t$ allow us to calculate the noisy data in one step?

In the forward process, the goal is to add noise incrementally to $\mathbf{x}_0$ over multiple time steps, resulting in a noisy version $\mathbf{x}_t$ at time step $t$. Each step of the process adds some Gaussian noise, which gradually turns the original data $\mathbf{x}_0$ into pure noise $\mathbf{x}_T$ over $T$ steps.

The forward process can be described step-by-step like this:

$$
q(\mathbf{x}_t | \mathbf{x}_{t-1}) = \mathcal{N}(\mathbf{x}_t; \sqrt{\alpha_t} \mathbf{x}_{t-1}, (1-\alpha_t) \mathbf{I})
$$

which means:
- $\mathbf{x}_t$ is a Gaussian sample with a mean of $\sqrt{\alpha_t} \mathbf{x}_{t-1}$ (scaled down from the previous step) and a variance of $(1-\alpha_t) \mathbf{I}$ (the added noise).

If we were to apply this process recursively, from $t = 1$ to $t = T$, the data would become noisier and noisier. But rather than doing this recursive process manually (one step at a time), we can jump directly to any time step $t$ using the closed-form expression of $\mathbf{x}_t$ in terms of $\mathbf{x}_0$.

This expression is:

$$
q(\mathbf{x}_t | \mathbf{x}_0) = \mathcal{N}(\mathbf{x}_t; \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t) \mathbf{I})
$$

- $\sqrt{\bar{\alpha}_t} \mathbf{x}_0$: This is the part of the original data that remains after $t$ steps. The scaling factor $\bar{\alpha}_t$ gets smaller over time because more noise is added, so the contribution of $\mathbf{x}_0$ to $\mathbf{x}_t$ diminishes.
- $(1 - \bar{\alpha}_t) \mathbf{I}$: This is the cumulative noise that has been added up to step $t$. It grows as $t$ increases, eventually turning $\mathbf{x}_t$ into pure noise.

### How does this allow for one-step calculation?

Instead of iterating through each time step and adding noise gradually (which would be computationally expensive), the closed-form expression lets us compute $\mathbf{x}_t$ directly from $\mathbf{x}_0$ using just the cumulative term $\bar{\alpha}_t$.

#### Example:

Imagine you start with clean data $\mathbf{x}_0$. If you want to compute the noisy data at step $t = 100$, instead of applying noise sequentially 100 times, you can jump straight to $t = 100$ by using:

$$
\mathbf{x}_t = \sqrt{\bar{\alpha}_t} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_t} \boldsymbol{\epsilon}
$$

where $\boldsymbol{\epsilon}$ is standard Gaussian noise $\mathcal{N}(0, I)$.

This works because $\bar{\alpha}_t$ captures the total effect of the cumulative noise addition over all steps up to $t$.

Let's define the following:
- $\mathbf{x}_0$ is the original data (e.g., an image).
- The noise schedule is given by $\beta_1, \beta_2, \beta_3$, and $\alpha_t = 1 - \beta_t$.
- The cumulative product $\bar{\alpha}_t$ is $\prod_{s=1}^{t} \alpha_s$.

### Step 1: From $t = 0$ to $t = 1$
We start with the clean data $\mathbf{x}_0$. The forward process at $t = 1$ is given by:
$$
q(\mathbf{x}_1 | \mathbf{x}_0) = \mathcal{N}(\mathbf{x}_1; \sqrt{\alpha_1} \mathbf{x}_0, \beta_1 \mathbf{I})
$$
This means:
- $\mathbf{x}_1$ is a Gaussian distribution with mean $\sqrt{\alpha_1} \mathbf{x}_0$ (a scaled version of the original data) and variance $\beta_1$, which adds some Gaussian noise.

The noisy data $\mathbf{x}_1$ is a mixture of the original data $\mathbf{x}_0$ and some added noise:
$$
\mathbf{x}_1 = \sqrt{\alpha_1} \mathbf{x}_0 + \sqrt{\beta_1} \boldsymbol{\epsilon}_1
$$
where $\boldsymbol{\epsilon}_1 \sim \mathcal{N}(0, I)$ is Gaussian noise.

### Step 2: From $t = 1$ to $t = 2$
Next, we add more noise to $\mathbf{x}_1$. The forward process at $t = 2$ is:
$$
q(\mathbf{x}_2 | \mathbf{x}_1) = \mathcal{N}(\mathbf{x}_2; \sqrt{\alpha_2} \mathbf{x}_1, \beta_2 \mathbf{I})
$$
Now we generate $\mathbf{x}_2$ by adding noise to $\mathbf{x}_1$:
$$
\mathbf{x}_2 = \sqrt{\alpha_2} \mathbf{x}_1 + \sqrt{\beta_2} \boldsymbol{\epsilon}_2
$$
Substituting $\mathbf{x}_1$ from the previous step:
$$
\mathbf{x}_2 = \sqrt{\alpha_2} \left( \sqrt{\alpha_1} \mathbf{x}_0 + \sqrt{\beta_1} \boldsymbol{\epsilon}_1 \right) + \sqrt{\beta_2} \boldsymbol{\epsilon}_2
$$
Simplifying:
$$
\mathbf{x}_2 = \sqrt{\alpha_2 \alpha_1} \mathbf{x}_0 + \sqrt{\alpha_2 \beta_1} \boldsymbol{\epsilon}_1 + \sqrt{\beta_2} \boldsymbol{\epsilon}_2
$$
So, $\mathbf{x}_2$ contains:
- The original data $\mathbf{x}_0$, scaled by $\sqrt{\alpha_2 \alpha_1}$,
- Noise from step 1, scaled by $\sqrt{\alpha_2 \beta_1}$,
- New noise added at step 2, $\boldsymbol{\epsilon}_2$, scaled by $\sqrt{\beta_2}$.

### Step 3: From $t = 2$ to $t = 3$
Now we repeat the process again, adding noise to $\mathbf{x}_2$. The forward process at $t = 3$ is:
$$
q(\mathbf{x}_3 | \mathbf{x}_2) = \mathcal{N}(\mathbf{x}_3; \sqrt{\alpha_3} \mathbf{x}_2, \beta_3 \mathbf{I})
$$
So we generate $\mathbf{x}_3$ by:
$$
\mathbf{x}_3 = \sqrt{\alpha_3} \mathbf{x}_2 + \sqrt{\beta_3} \boldsymbol{\epsilon}_3
$$
Substitute $\mathbf{x}_2$ from the previous step:
$$
\mathbf{x}_3 = \sqrt{\alpha_3} \left( \sqrt{\alpha_2 \alpha_1} \mathbf{x}_0 + \sqrt{\alpha_2 \beta_1} \boldsymbol{\epsilon}_1 + \sqrt{\beta_2} \boldsymbol{\epsilon}_2 \right) + \sqrt{\beta_3} \boldsymbol{\epsilon}_3
$$
Simplifying:
$$
\mathbf{x}_3 = \sqrt{\alpha_3 \alpha_2 \alpha_1} \mathbf{x}_0 + \sqrt{\alpha_3 \alpha_2 \beta_1} \boldsymbol{\epsilon}_1 + \sqrt{\alpha_3 \beta_2} \boldsymbol{\epsilon}_2 + \sqrt{\beta_3} \boldsymbol{\epsilon}_3
$$
Now $\mathbf{x}_3$ contains:
- The original data $\mathbf{x}_0$, scaled by $\sqrt{\alpha_3 \alpha_2 \alpha_1}$,
- Noise added at step 1, scaled by $\sqrt{\alpha_3 \alpha_2 \beta_1}$,
- Noise added at step 2, scaled by $\sqrt{\alpha_3 \beta_2}$,
- New noise added at step 3, $\boldsymbol{\epsilon}_3$, scaled by $\sqrt{\beta_3}$.

### Using $\bar{\alpha}_t$ to Jump Directly from $\mathbf{x}_0$ to $\mathbf{x}_3$

Instead of computing the noise addition step-by-step, we can jump directly to $t = 3$ using the closed-form expression with $\bar{\alpha}_3$.

The cumulative product of the $\alpha_t$'s up to $t = 3$ is:
$$
\bar{\alpha}_3 = \alpha_1 \alpha_2 \alpha_3
$$

The formula for $\mathbf{x}_3$ is:
$$
q(\mathbf{x}_3 | \mathbf{x}_0) = \mathcal{N}(\mathbf{x}_3; \sqrt{\bar{\alpha}_3} \mathbf{x}_0, (1 - \bar{\alpha}_3) \mathbf{I})
$$

So, we can directly compute $\mathbf{x}_3$ as:
$$
\mathbf{x}_3 = \sqrt{\bar{\alpha}_3} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_3} \boldsymbol{\epsilon}
$$
where $\boldsymbol{\epsilon} \sim \mathcal{N}(0, I)$.

#### What does this mean?
- Instead of calculating noise step by step from $t = 0$ to $t = 3$, we can **directly compute** $\mathbf{x}_3$ by just applying a scaling factor to $\mathbf{x}_0$ and adding Gaussian noise $\boldsymbol{\epsilon}$, with variance determined by $1 - \bar{\alpha}_3$.
- $\bar{\alpha}_3$ encodes the cumulative scaling of the original data after 3 steps.
- $1 - \bar{\alpha}_3$ represents the cumulative noise added after 3 steps.

#### Reverse Process (Denoising)

The reverse process is where we attempt to **undo** the forward process. While the forward process gradually turns clean data $\mathbf{x}_0$ into noise $\mathbf{x}_T$, the reverse process tries to reconstruct the original data $\mathbf{x}_0$ from noisy samples $\mathbf{x}_T$.

However, we don't know the true reverse process directly (i.e., how to denoise perfectly). Instead, we **train a neural network** to learn this reverse process step-by-step by predicting the noise at each step.

#### Goal of the reverse process
We want to model the reverse transitions $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$, which allow us to go backward from $\mathbf{x}_T$ to $\mathbf{x}_0$. This is modeled as another Gaussian distribution:
$$
p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}(\mathbf{x}_{t-1}; \boldsymbol{\mu}_\theta(\mathbf{x}_t, t), \sigma^2_t \mathbf{I})
$$
Where:
- $\boldsymbol{\mu}_\theta(\mathbf{x}_t, t)$ is the mean of the Gaussian distribution (which will be predicted by the neural network).
- $\sigma^2_t$ is the variance, which can be fixed or learned, but is often kept constant for simplicity.

#### Intuition Behind the Reverse Process

At each step in the reverse process, the neural network learns to predict the mean $\boldsymbol{\mu}_\theta(\mathbf{x}_t, t)$ — which is essentially the "cleaner" version of $\mathbf{x}_t$ with some noise removed — allowing the model to step backward toward the original data $\mathbf{x}_0$.

Here's the step-by-step intuition:
1. At $t = T$, we start with a noisy sample $\mathbf{x}_T$ (essentially random Gaussian noise).
2. The neural network predicts how to denoise $\mathbf{x}_T$ into a less noisy sample $\mathbf{x}_{T-1}$.
3. This process is repeated in a step-by-step manner, denoising $\mathbf{x}_t$ to obtain $\mathbf{x}_{t-1}$, until eventually reaching $\mathbf{x}_0$.

#### Key Insight: Predicting the Noise

A key insight in DDPMs is that instead of predicting $\mathbf{x}_{t-1}$ directly, the neural network is trained to **predict the noise $\boldsymbol{\epsilon}_t$ added at step $t$**.

Why? Because:
- In the forward process, each $\mathbf{x}_t$ is a combination of the clean data $\mathbf{x}_0$ and Gaussian noise $\boldsymbol{\epsilon}$, as we saw earlier.
- If the neural network can predict the noise $\boldsymbol{\epsilon}_t$, we can then subtract it from $\mathbf{x}_t$ to get an estimate of the clean data.

This leads to the formulation:
$$
\mathbf{x}_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( \mathbf{x}_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \boldsymbol{\epsilon}_\theta(\mathbf{x}_t, t) \right) + \sigma_t \mathbf{z}
$$
Where:
- $\boldsymbol{\epsilon}_\theta(\mathbf{x}_t, t)$ is the noise predicted by the neural network at step $t$.
- $\mathbf{z}$ is Gaussian noise added for randomness, usually sampled from $\mathcal{N}(0, I)$.
- $\sigma_t$ controls the variance in this step.

#### Training the Neural Network

#### Objective of Training
The neural network is trained to minimize the difference between the **true noise $\boldsymbol{\epsilon}_t$** (which was added during the forward process) and the noise it **predicts $\boldsymbol{\epsilon}_\theta(\mathbf{x}_t, t)$** at each time step $t$.

This gives rise to the **loss function** for training the network, which is typically a simple mean squared error (MSE) between the true noise and the predicted noise:
$$
L_\text{simple} = \mathbb{E}_{\mathbf{x}_0, t, \boldsymbol{\epsilon}} \left[ \left\| \boldsymbol{\epsilon} - \boldsymbol{\epsilon}_\theta(\mathbf{x}_t, t) \right\|^2 \right]
$$
Where:
- $\boldsymbol{\epsilon}$ is the true noise sampled from $\mathcal{N}(0, I)$ during the forward process.
- $\boldsymbol{\epsilon}_\theta(\mathbf{x}_t, t)$ is the noise predicted by the neural network at time step $t$.
- The expectation is taken over the data $\mathbf{x}_0$, time steps $t$, and the noise $\boldsymbol{\epsilon}$.

#### How does training work?
- **Input**: During training, the network is provided with noisy samples $\mathbf{x}_t$ (generated using the forward process) and the time step $t$.
- **Output**: The network is trained to output the noise $\boldsymbol{\epsilon}_\theta(\mathbf{x}_t, t)$ that was added to generate $\mathbf{x}_t$.
- **Loss**: The loss function compares the predicted noise to the true noise $\boldsymbol{\epsilon}$, and the network adjusts its parameters to minimize this error over time.

#### Variational Inference in DDPMs

In DDPMs, **variational inference** helps us frame the reverse process as an approximation to the true reverse distribution. The key goal is to train the model to learn this reverse process in a way that maximizes the likelihood of the data. This involves minimizing the **KL divergence** between the learned reverse distribution and the true reverse distribution of the forward process.

Here’s the breakdown:

#### What are we trying to achieve?
- We want to **learn the reverse process** $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$ that moves us from noisy data $ \mathbf{x}_T $ back to the original clean data $ \mathbf{x}_0 $.
- Ideally, we would use the **true reverse distribution** $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ to perfectly denoise the data.
- However, since $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ is intractable (we can't compute it exactly), we approximate it using a parametric distribution $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$, which is modeled by a neural network.

To train the neural network, we want to **minimize the difference** between the true reverse distribution $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ and the approximate distribution $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$. This difference is measured using **KL divergence**.

#### KL Divergence: Minimizing the Difference Between Distributions

The **KL divergence** measures how different one probability distribution is from another. In our case, we want to minimize the KL divergence between the true reverse distribution $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ and the learned reverse process $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$.

The KL divergence between two distributions $q$ and $p$ is defined as:
$$
\text{KL}(q \| p) = \mathbb{E}_q \left[ \log \frac{q(\mathbf{x})}{p(\mathbf{x})} \right]
$$

In the context of DDPMs, we are specifically minimizing the KL divergence between $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ and $p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t)$, which can be written as:
$$
\text{KL}\left( q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) \| p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) \right)
$$

This term encourages the model to learn a reverse process that closely matches the true (but intractable) reverse process.

#### Evidence Lower Bound (ELBO): Objective for Training

The **ELBO** (Evidence Lower Bound) is a variational inference technique that allows us to maximize the likelihood of the data by minimizing the KL divergence between the model's learned distribution and the true distribution.

In DDPMs, the **log likelihood** of the data can be decomposed into a series of steps using variational inference. This gives us the **ELBO**, which we aim to maximize during training. The ELBO can be written as:
$$
\log p_\theta(\mathbf{x}_0) \geq \mathbb{E}_q \left[ \log p_\theta(\mathbf{x}_0 | \mathbf{x}_1) \right] - \sum_{t=2}^{T} \mathbb{E}_q \left[ \text{KL}\left( q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) \| p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) \right) \right]
$$

#### What does this mean?
- **$\log p_\theta(\mathbf{x}_0)$**: This is the log likelihood of the data, which is what we want to maximize.
- The **ELBO** breaks this down into:
  - **A reconstruction term**: $ \mathbb{E}_q \left[ \log p_\theta(\mathbf{x}_0 | \mathbf{x}_1) \right] $ is a reconstruction term that measures how well the model can reconstruct the original data from the noisy data at step $ t = 1 $.
  - **KL terms**: The sum of KL divergences between the true reverse process and the model’s reverse process at each step $ t $, ensuring the model’s reverse steps approximate the true reverse process.

#### Breaking Down the ELBO (DDPM-Specific Objective)

In DDPMs, the ELBO consists of a **sum of KL divergence terms** over all time steps $t$, and a term that ensures the final step (reconstruction of $ \mathbf{x}_0 $ from $ \mathbf{x}_1 $) is accurate.

Let's break it down:

1. **KL divergence at each step $ t $**: For each time step $ t $, we compute the KL divergence between the true reverse process $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ and the learned reverse process $ p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) $. This ensures the model learns a good approximation of the reverse process.

2. **Reconstruction at $ t = 1 $**: At the last time step of the reverse process, $ p_\theta(\mathbf{x}_0 | \mathbf{x}_1) $ (pronounced **"p theta of x naught given x sub 1"**) measures how well the model can reconstruct the original data $ \mathbf{x}_0 $ from the slightly noisy $ \mathbf{x}_1 $. This term is typically modeled as a Gaussian distribution.

3. **Variance schedule**: In addition to learning the mean $ \boldsymbol{\mu}_\theta $, the model can also learn the variance $ \sigma^2_t $ of the reverse process. This ensures the model properly handles uncertainty during the reverse process.

#### Simplifying the Training Loss (Connecting to the MSE Loss)

While the **ELBO** provides a complete theoretical framework, in practice, the training loss in DDPMs is often simplified to the **mean squared error (MSE)** between the true noise $ \epsilon_t $ and the noise predicted by the model $ \epsilon_\theta(\mathbf{x}_t, t) $, as we saw earlier:
$$
L_\text{simple} = \mathbb{E}_{\mathbf{x}_0, t, \epsilon} \left[ \| \epsilon - \epsilon_\theta(\mathbf{x}_t, t) \|^2 \right]
$$
(pronounced: **"L simple equals the expectation over x naught, t, and epsilon of the squared difference between epsilon and epsilon theta of x sub t and t"**)

This simpler loss is derived by simplifying the KL divergence terms in the ELBO and assuming the reverse process is Gaussian. The MSE loss works well because it directly trains the neural network to predict the noise added during the forward process, which is key for denoising.

#### What is $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$?

The distribution $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ (pronounced: **"q of x sub t minus 1 given x sub t and x naught"**) represents the **true reverse process** that describes how we can move from noisy data $ \mathbf{x}_t $ (at time step $ t $) to less noisy data $ \mathbf{x}_{t-1} $ (at time step $ t-1 $).

In simpler terms, this is the **ideal denoising step** that perfectly tells us how to reverse the noise added at each time step during the forward process.

#### Why do we want to compute $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $?

If we could compute this true reverse distribution $q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ exactly, we could perfectly reverse the forward diffusion process. This would allow us to denoise the data step-by-step without any approximation, and we would be able to recover the clean data $ \mathbf{x}_0 $ from a noisy sample $ \mathbf{x}_T $.

However, the problem is that **this distribution is intractable**.

#### Why is $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ intractable?

To understand why $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ is intractable, let's break it down into two main reasons:

#### (a) Complexity of the Conditionals
$q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0)$ involves **two unknowns** — both $ \mathbf{x}_t $ (the noisy data) and $ \mathbf{x}_0 $ (the original data). Computing this conditional distribution exactly would require us to model the complex joint distribution of $ \mathbf{x}_t $ and $ \mathbf{x}_0 $ across all time steps of the forward process. This involves high-dimensional integrations over the latent variables, which are extremely difficult to compute analytically.

For example, in the forward process, $ \mathbf{x}_t $ is generated through a sequence of noisy transformations starting from $ \mathbf{x}_0 $:
$$
q(\mathbf{x}_t | \mathbf{x}_0) = \mathcal{N}(\mathbf{x}_t; \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1 - \bar{\alpha}_t) \mathbf{I})
$$

To compute the true reverse process $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $, we would need to account for **all the possible ways** that noise could have been added at each prior step in the forward process to create $ \mathbf{x}_t $. This requires solving a complex probabilistic relationship involving **many interacting variables**, which is computationally intractable.

#### (b) Marginalization Over $ \mathbf{x}_0 $
Another challenge is that the true reverse process depends on the original clean data $ \mathbf{x}_0 $, which we don’t have access to during the reverse process. This means that we need to **marginalize over $ \mathbf{x}_0 $** (integrate out $ \mathbf{x}_0 $) in order to compute $ q(\mathbf{x}_{t-1} | \mathbf{x}_t) $.

The true reverse distribution is given by:
$$
q(\mathbf{x}_{t-1} | \mathbf{x}_t) = \int q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) p(\mathbf{x}_0 | \mathbf{x}_t) \, d\mathbf{x}_0
$$

This means we need to compute the probability of $ \mathbf{x}_{t-1} $ given $ \mathbf{x}_t $, by averaging over all possible values of $ \mathbf{x}_0 $. But marginalizing over $ \mathbf{x}_0 $ (the clean data) is a high-dimensional integration problem that we cannot solve analytically in closed form, especially for large and complex datasets like images.

#### Why Can't We Approximate $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ with Simple Distributions?

The true reverse process $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ is a complex distribution because it describes how to **denoise the data** at each time step in a way that depends on the entire history of noise added up to that point. This complexity comes from the fact that:
- The noise is added in small, incremental steps, and how the noise "should be removed" at step $ t $ depends on all prior steps.
- The forward process involves adding noise in a probabilistic, stochastic way, which makes it difficult to write down an exact form for how to reverse this process.

If we tried to approximate the true reverse process using simple distributions, we would miss these dependencies, and the reverse process would become inaccurate.

#### Why Do We Use $ p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) $ Instead?

Since $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ is intractable, we instead **learn an approximation** using a neural network. This gives us the **learned reverse distribution** $ p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) $ (pronounced **"p theta of x sub t minus 1 given x sub t"**), which is modeled as a Gaussian distribution.

The learned reverse process $ p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) $ is parameterized by the neural network $ \epsilon_\theta(\mathbf{x}_t, t) $ (which predicts the noise), and is trained to approximate the true reverse process by minimizing the KL divergence between $ q(\mathbf{x}_{t-1} | \mathbf{x}_t, \mathbf{x}_0) $ and $ p_\theta(\mathbf{x}_{t-1} | \mathbf{x}_t) $.

In [ ]:
# Standard library imports
import math
import os
import time
import shutil
import logging
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter, itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

# Third-party library imports
import fastcore.all as fc
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import gzip
import pickle

# torch imports
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch import tensor, nn, optim
from torch.utils.data import DataLoader, default_collate
from torch.nn import init
from torch.optim import lr_scheduler
from torcheval.metrics import MulticlassAccuracy

# dataset imports
import datasets
from datasets import load_dataset, load_dataset_builder

# miniai imports
from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *
from miniai.init import *
from miniai.sgd import *
from miniai.resnet import *
from miniai.augment import *

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'
logging.disable(logging.WARNING)

In [ ]:
x,y = 'image', 'label'
dsd = load_dataset('fashion_mnist')

In [ ]:
@inplace
def transformi(b):
    b[x] = [TF.resize(TF.to_tensor(o), (32,32)) for o in b[x]] # resize the 28x28 images to 32x32 to make it simpler for model's architecture 

In [ ]:
set_seed(42)
bs = 128
tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=8)
dt = dls.train
xb,yb = next(iter(dt))
xb.shape,yb[:10]